In [28]:
from SmartApi.smartConnect import SmartConnect
import pyotp
import pandas as pd
import requests
import pandas as pd
from datetime import datetime
from datetime import datetime, timedelta
import pandas_ta as ta


api_key = 'C46AImmE'
clientId = 'M507690'
pwd = '4663'
smartApi = SmartConnect(api_key)
token = "4ACNNSIZL6EV6UQ2ACSI23H5VQ"
totp=pyotp.TOTP(token).now()
correlation_id = "abc123"

# login api call

data = smartApi.generateSession(clientId, pwd, totp)
# print(data)
authToken = data['data']['jwtToken']
refreshToken = data['data']['refreshToken']

# fetch the feedtoken
feedToken = smartApi.getfeedToken()

# fetch User Profile
res = smartApi.getProfile(refreshToken)
smartApi.generateToken(refreshToken)
res=res['data']['exchanges']

#fetch User Profile
userProfile= smartApi.getProfile(refreshToken)
User= pd.DataFrame(userProfile)
User = User['data']['name']
User
print(f"Welcome {User} to the World of SMART API BY ANGEL.....!!")
print(f"Login Sucessfull....!!")


[I 251217 11:58:49 smartConnect:124] in pool


Welcome MANA  BHATTACHARYYA to the World of SMART API BY ANGEL.....!!
Login Sucessfull....!!


In [29]:
# ---- NIFTY 50 Token (Angel One Fixed Token) ----
NIFTY_TOKEN = "26000"      # NIFTY 50 Index
EXCHANGE = "NSE"

# ---- Fetch LTP ----
ltp_data = smartApi.ltpData(
    exchange=EXCHANGE,
    tradingsymbol="NIFTY",
    symboltoken=NIFTY_TOKEN
)

print("📈 NIFTY 50 LTP:", ltp_data['data']['ltp'])

ATM = round(ltp_data['data']['ltp'] / 50) * 50
print("📈 NIFTY 50 SPOT Price:", ATM)

ltp =  ltp_data['data']['ltp']
prev_close = ltp_data['data']['close']

change = ltp - prev_close
pct_change = (change / prev_close) * 100

print(f"NIFTY Change: {change:.2f}")
print(f"NIFTY % Change: {pct_change:.2f}%")

📈 NIFTY 50 LTP: 25828.0
📈 NIFTY 50 SPOT Price: 25850
NIFTY Change: -32.10
NIFTY % Change: -0.12%


In [30]:
url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
instruments = pd.DataFrame(requests.get(url).json())

expiry = instruments[(instruments['name']=='NIFTY') &  (instruments['instrumenttype']=='OPTIDX') &  (instruments['exch_seg']=='NFO')]["expiry"].unique()
expiry

today = datetime.today()

dates = pd.to_datetime(expiry, format="%d%b%Y")
next_expiry = dates[dates > today].min()
print("Next Expiry:", next_expiry.strftime("%d%b%y").upper())

Next Expiry: 23DEC25


In [31]:
expiry_next = next_expiry.strftime("%d%b%y").upper()

def get_option_symbol(expiry_next, strike, opt_type):
    return f"NIFTY{expiry_next}{strike}{opt_type}"

ce_symbol = get_option_symbol(expiry_next, ATM, "CE")
pe_symbol = get_option_symbol(expiry_next, ATM, "PE")

print("CE Symbol:", ce_symbol)
print("PE Symbol:", pe_symbol)

CE Symbol: NIFTY23DEC2525850CE
PE Symbol: NIFTY23DEC2525850PE


In [32]:
def get_option_token(symbol):
    row = instruments[instruments["symbol"] == symbol]
    if row.empty:
        raise Exception(f"Token not found for {symbol}")
    return row.iloc[0]["token"]

ce_token = get_option_token(ce_symbol)
pe_token = get_option_token(pe_symbol)

print("CE Token:", ce_token)
print("PE Token:", pe_token)

CE Token: 56994
PE Token: 56995


In [33]:
def fetch_5min_candles(token):
    to_date = datetime.now()
    from_date = to_date - timedelta(days=3)

    response = smartApi.getCandleData({
        "exchange": "NFO",
        "symboltoken": token,
        "interval": "THREE_MINUTE",
        "fromdate": from_date.strftime("%Y-%m-%d %H:%M"),
        "todate": to_date.strftime("%Y-%m-%d %H:%M")
    })

    if not response or "data" not in response or response["data"] is None:
        raise Exception("No candle data")

    df = pd.DataFrame(
        response["data"],
        columns=["datetime", "open", "high", "low", "close", "volume"]
    )

    # 🔑 CRITICAL FIX
    df["datetime"] = pd.to_datetime(df["datetime"])
    df.set_index("datetime", inplace=True)
    df.sort_index(inplace=True)

    return df



def apply_indicators(df):
    df = df.copy()

    df["EMA14"] = ta.ema(df["close"], length=5)
    df["EMA20"] = ta.ema(df["close"], length=10)
    df["RSI"] = ta.rsi(df["close"], length=14)

    df["VWAP"] = ta.vwap(
        high=df["high"],
        low=df["low"],
        close=df["close"],
        volume=df["volume"]
    )

    df.dropna(inplace=True)
    return df

pe_df = fetch_5min_candles(ce_token)
pe_df = fetch_5min_candles(pe_token)

pe_df = apply_indicators(pe_df)
pe_df = apply_indicators(pe_df)

In [34]:
latest = pe_df.tail(5)
latest

,open,high,low,close,volume,EMA14,EMA20,RSI,VWAP
datetime,,,,,,,,,
2025-12-17 11:45:00+05:30,111.85,112.80,105.50,106.35,18286,112.049494,113.833129,41.721222,107.811815
2025-12-17 11:48:00+05:30,106.00,109.50,105.60,106.05,11925,110.049663,112.418015,41.450649,107.805676
2025-12-17 11:51:00+05:30,106.05,112.65,105.95,109.70,27551,109.933108,111.923830,46.036153,107.835425
2025-12-17 11:54:00+05:30,109.70,114.85,109.00,114.55,17230,111.472072,112.401316,51.474519,107.891531
2025-12-17 11:57:00+05:30,114.80,116.20,110.80,115.90,13215,112.948048,113.037440,52.897459,107.946601


In [35]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = go.Figure()
pe_df = pe_df.reset_index()
# ---- Price & Indicators ----
fig.add_trace(
    go.Scatter(
        x=pe_df.index,
        y=pe_df["close"],
        name="Close",
        line=dict(color="yellow", width=4)
    )
)

fig.add_trace(
    go.Scatter(
        x=pe_df.index,
        y=pe_df["EMA14"],
        name="EMA14",
        line=dict(color="cyan", width=2.5)
    )
)

fig.add_trace(
    go.Scatter(
        x=pe_df.index,
        y=pe_df["EMA20"],
        name="EMA20",
        line=dict(color="magenta", width=2.5)
    )
)

fig.add_trace(
    go.Scatter(
        x=pe_df.index,
        y=pe_df["VWAP"],
        name="VWAP",
        line=dict(color="orange", width=5, dash="dot")
    )
)



# Layout
fig.update_layout(
    title=f"{ce_symbol}:  EMA+VWAP+RSI Graph:",
    width=2000,
    height=600,
    xaxis=dict(title='Date'),
    yaxis=dict(title='Change & Net Money Flow'),
    yaxis2=dict(title='NIFTY50 LTP', overlaying='y', side='right'),
    legend=dict(x=0.9, y=1.15)
)


In [36]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    row_heights=[0.7, 0.3],
    subplot_titles=("Price Action", "RSI (14)")
)


fig.add_trace(
    go.Scatter(x=pe_df.index, y=pe_df["close"],
               name="Close", line=dict(color="yellow", width=2)),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=pe_df.index, y=pe_df["EMA14"],
               name="EMA14", line=dict(color="cyan", width=1.2)),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=pe_df.index, y=pe_df["EMA20"],
               name="EMA20", line=dict(color="magenta", width=1.2)),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=pe_df.index, y=pe_df["VWAP"],
               name="VWAP", line=dict(color="orange", width=1.2, dash="dot")),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=pe_df.index, y=pe_df["RSI"],
               name="RSI", line=dict(color="lightgreen", width=1.5)),
    row=2, col=1
)

fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
fig.add_hline(y=50, line_dash="dot", line_color="gray", row=2, col=1)

fig.update_layout(
    title="📊 ATM Option – EMA | VWAP | RSI",
    height=550,
    template="plotly_dark",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    margin=dict(l=20, r=20, t=40, b=20)
)

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="RSI", row=2, col=1, range=[0, 100])

In [37]:
df = pe_df.copy()

df["signal"] = 0
df.loc[
    (df["EMA14"] > df["EMA20"]) &
    (df["close"] > df["VWAP"]) &
    (df["RSI"] > 60),
    "signal"
] = 1   # BUY

df.loc[
    (df["EMA14"] < df["EMA20"]) &
    (df["close"] < df["VWAP"]) &
    (df["RSI"] < 35),
    "signal"
] = -1  # SELL


trades = []
position = None
entry_price = None

for i in range(len(df)):
    if df["signal"].iloc[i] == 1 and position is None:
        position = "BUY"
        entry_price = df["close"].iloc[i]
        entry_time = df.index[i]

    elif df["signal"].iloc[i] == -1 and position == "BUY":
        exit_price = df["close"].iloc[i]
        exit_time = df.index[i]

        pnl = exit_price - entry_price

        trades.append({
            "entry_time": entry_time,
            "exit_time": exit_time,
            "entry": entry_price,
            "exit": exit_price,
            "pnl": pnl
        })

        position = None



trades_df = pd.DataFrame(trades)

if not trades_df.empty:
    trades_df["result"] = trades_df["pnl"].apply(lambda x: 1 if x > 0 else 0)
    success_rate = round(trades_df["result"].mean() * 100, 2)
else:
    success_rate = 0.0


buy_signals = df[df["signal"] == 1]
sell_signals = df[df["signal"] == -1]


from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    row_heights=[0.7, 0.3],
    vertical_spacing=0.08,
    subplot_titles=(
        f"📈 ATM Option Strategy | Success Rate: {success_rate}%",
        "RSI (14)"
    )
)

# ----- PRICE ROW -----
fig.add_trace(go.Scatter(
    x=df.index, y=df["close"],
    name="Close", line=dict(color="yellow", width=2)
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df["EMA14"],
    name="EMA14", line=dict(color="cyan", width=1)
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df["EMA20"],
    name="EMA20", line=dict(color="magenta", width=1)
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df["VWAP"],
    name="VWAP", line=dict(color="orange", dash="dot")
), row=1, col=1)

# ----- BUY / SELL MARKERS -----
fig.add_trace(go.Scatter(
    x=buy_signals.index,
    y=buy_signals["close"],
    mode="markers",
    name="Buy",
    marker=dict(symbol="triangle-up", size=14, color="green")
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=sell_signals.index,
    y=sell_signals["close"],
    mode="markers",
    name="Sell",
    marker=dict(symbol="triangle-down", size=14, color="red")
), row=1, col=1)

# ----- RSI ROW -----
fig.add_trace(go.Scatter(
    x=df.index, y=df["RSI"],
    name="RSI", line=dict(color="lightgreen")
), row=2, col=1)

fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
fig.add_hline(y=50, line_dash="dot", line_color="gray", row=2, col=1)

# ----- LAYOUT -----
fig.update_layout(
    template="plotly_dark",
    height=600,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, x=1, xanchor="right"),
    margin=dict(l=20, r=20, t=50, b=20)
)

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="RSI", range=[0, 100], row=2, col=1)




In [38]:
print(f"Strategy Success Rate: {success_rate}%")
total_trades = len(trades_df)
winning_trades = trades_df["result"].sum()

print("Total Trades:", total_trades)
print("Winning Trades:", winning_trades)
print(f"Success Rate: {success_rate}%")

Strategy Success Rate: 50.0%
Total Trades: 2
Winning Trades: 1
Success Rate: 50.0%


In [39]:
trades_df

,entry_time,exit_time,entry,exit,pnl,result
0,107,172,86.20,94.1,7.90,1
1,208,234,106.35,83.2,-23.15,0
